<a href="https://colab.research.google.com/github/gabrielfernandorey/EDVAI/blob/main/PySpark/PySpark_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark

### Instalación y carga de Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=a9461b83b1960e8ee24bbce11b50f2e9a14ced2463da3b80cb0c8c8ff80a13bd
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

### Librerías necesarias

In [4]:
from pyspark.sql.types import StringType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator

from datetime import datetime
import numpy as np

### Importamos datos

In [5]:
!wget https://data-engineer-edvai.s3.amazonaws.com/yellow_tripdata_2021-01.csv

--2023-04-17 12:50:28--  https://data-engineer-edvai.s3.amazonaws.com/yellow_tripdata_2021-01.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 54.231.204.9, 3.5.11.119, 54.231.172.49, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|54.231.204.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125981363 (120M) [text/csv]
Saving to: ‘yellow_tripdata_2021-01.csv’

yellow_tripdata_202 100%[===================>] 120.14M  30.8MB/s    in 4.5s    

2023-04-17 12:50:33 (26.7 MB/s) - ‘yellow_tripdata_2021-01.csv’ saved [125981363/125981363]



In [ ]:
# Con esta alternativa algunos tipos de datos no se importan de forma adecuada
# df = spark.read.option("header","true").csv("*.csv")

In [6]:
# Con esta alternativa los tipos de datos se importan de forma adecuada (siempre verificar!)
df = spark.read.option("header","true").option("inferSchema", "true") .csv("*.csv")

In [7]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [8]:
df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

### Insertar en la tabla payments (VendorID, tpep_pickup_datetetime, payment_type, total_amount) Solamente los pagos con tarjeta de crédito

In [9]:
# Creamos vista
df.createOrReplaceTempView("yellow_tripdata")

In [10]:
df_5 = spark.sql("select VendorId, cast(tpep_pickup_datetime as date), cast(payment_type as int), total_amount from yellow_tripdata where payment_type = 1")

In [ ]:
df_5.show(10)

+--------+--------------------+------------+------------+
|VendorId|tpep_pickup_datetime|payment_type|total_amount|
+--------+--------------------+------------+------------+
|       1|          2021-01-01|           1|       51.95|
|       1|          2021-01-01|           1|       36.35|
|       2|          2021-01-01|           1|       24.36|
|       1|          2021-01-01|           1|       14.15|
|       1|          2021-01-01|           1|       18.95|
|       2|          2021-01-01|           1|        24.3|
|       2|          2021-01-01|           1|       10.79|
|       2|          2021-01-01|           1|       14.16|
|       2|          2021-01-01|           1|        10.3|
|       2|          2021-01-01|           1|       12.09|
+--------+--------------------+------------+------------+
only showing top 10 rows



### Insertar en la tabla passengers (tpep_pickup_datetetime, passenger_count, total_amount) los registros cuya cantidad de pasajeros sea mayor a 2 y el total del viaje cueste más de 8 dólares.

In [11]:
df_6 = spark.sql("select cast(tpep_pickup_datetime as date) as tpep_pickup_date , passenger_count, total_amount from yellow_tripdata where passenger_count >2 and total_amount > 8")

In [12]:
df_6.show(10)

+----------------+---------------+------------+
|tpep_pickup_date|passenger_count|total_amount|
+----------------+---------------+------------+
|      2021-01-01|              3|        24.3|
|      2021-01-01|              5|       14.16|
|      2021-01-01|              5|         8.3|
|      2021-01-01|              3|         9.3|
|      2021-01-01|              4|        18.3|
|      2021-01-01|              4|        13.3|
|      2021-01-01|              3|        40.3|
|      2021-01-01|              5|        14.8|
|      2021-01-01|              3|       18.59|
|      2021-01-01|              3|       13.56|
+----------------+---------------+------------+
only showing top 10 rows



Los valores son aproximados pero no exactos al resultado real

### Insertar en la tabla tolls (tpep_pickup_datetetime, passenger_count, tolls_amount, total_amount) los registros que tengan pago de peajes mayores a 0.1 y cantidad de pasajeros mayores a 1.

In [13]:
df_7 = spark.sql("select cast(tpep_pickup_datetime as date) as tpep_pickup_date, passenger_count, tolls_amount, total_amount from yellow_tripdata where tolls_amount > 0.1 and passenger_count > 1 ")

In [14]:
df_7.show(10)

+----------------+---------------+------------+------------+
|tpep_pickup_date|passenger_count|tolls_amount|total_amount|
+----------------+---------------+------------+------------+
|      2021-01-01|              2|        6.12|       33.92|
|      2021-01-01|              2|        6.12|       59.42|
|      2021-01-01|              2|        6.12|       35.92|
|      2021-01-01|              6|        6.12|        40.1|
|      2021-01-01|              3|        6.12|        54.0|
|      2021-01-01|              2|         2.8|        34.1|
|      2021-01-01|              4|        6.12|       61.42|
|      2021-01-01|              4|        6.12|       51.42|
|      2021-01-01|              2|       11.75|       12.05|
|      2021-01-01|              6|        6.12|       71.42|
+----------------+---------------+------------+------------+
only showing top 10 rows



### Insertar en la tabla congestion (tpep_pickup_datetetime, passenger_count, congestion_surcharge, total_amount) los registros que hayan tenido congestión en los viajes en la fecha 2021-01-18

In [15]:
df_8 = spark.sql("select cast(tpep_pickup_datetime as date) as tpep_pickup_date, passenger_count, congestion_surcharge, total_amount from yellow_tripdata where cast(tpep_pickup_datetime as date) = '2021-01-18' and cast(congestion_surcharge as float) != 0 ")

In [16]:
df_8.show()

+----------------+---------------+--------------------+------------+
|tpep_pickup_date|passenger_count|congestion_surcharge|total_amount|
+----------------+---------------+--------------------+------------+
|      2021-01-18|              1|                 2.5|        10.8|
|      2021-01-18|              1|                 2.5|       16.56|
|      2021-01-18|              1|                 2.5|       11.16|
|      2021-01-18|              1|                 2.5|        11.3|
|      2021-01-18|              1|                 2.5|       21.23|
|      2021-01-18|              1|                 2.5|       12.96|
|      2021-01-18|              1|                 2.5|       13.87|
|      2021-01-18|              1|                 2.5|        14.8|
|      2021-01-18|              1|                 2.5|       14.14|
|      2021-01-18|              1|                 2.5|        20.8|
|      2021-01-18|              1|                 2.5|       25.87|
|      2021-01-18|              1|

### Insertar en la tabla distance (tpep_pickup_datetetime, passenger_count, trip_distance, total_amount) los registros de la fecha 2020-12-31 que hayan tenido solamente un pasajero (passenger_count = 1) y hayan recorrido más de 15 millas (trip_distance).

In [19]:
df_9 = spark.sql("select cast(tpep_pickup_datetime as date) as tpep_pickup_date, passenger_count, trip_distance, total_amount from yellow_tripdata where cast(tpep_pickup_datetime as date) = '2020-12-31' and passenger_count = 1 and trip_distance > 15 ")

In [20]:
df_9.show()

+----------------+---------------+-------------+------------+
|tpep_pickup_date|passenger_count|trip_distance|total_amount|
+----------------+---------------+-------------+------------+
|      2020-12-31|              1|        17.96|        53.3|
+----------------+---------------+-------------+------------+

